In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

/home/pedro/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [11]:
from backtesting.test import SMA, GOOG

In [3]:
class SmaCross(Strategy):
    n1=0
    n2=0

        
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            
    def setInitialParams(self,valor1, valor2):
        self.n1 = valor1;
        self.n2 = valor2;

In [4]:
SmaCross.setInitialParams(SmaCross,200, 400)
bt = Backtest(GOOG, SmaCross,cash=10000,exclusive_orders=True)

In [6]:
maiorRetorno = {}
maoirAcerto = {}
tamanho = 10
for s1 in range(tamanho):
    for s2 in range(tamanho):
        print("testando nos valores ", s1, " e ", s2)
        SmaCross.setInitialParams(SmaCross,s1, s2)
        bt = Backtest(GOOG, SmaCross,cash=100000,exclusive_orders=True)
        output = bt.run()
        if maiorRetorno=={}:
            maiorRetorno = {"sma1": s1, "sma2":s2, "output":output}
        if maoirAcerto=={}:
            maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
        if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
            maiorRetorno = {"sma1": s1, "sma2":s2, "output":output}
        if output['Win Rate [%]'] > maiorRetorno["output"]['Win Rate [%]'] and output['# Trades'] > 9:
            maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
            
print(maiorRetorno)
print(maoirAcerto)
        

testando nos valores  0  e  0
testando nos valores  0  e  1
testando nos valores  0  e  2
testando nos valores  0  e  3
testando nos valores  0  e  4
testando nos valores  0  e  5
testando nos valores  0  e  6
testando nos valores  0  e  7
testando nos valores  0  e  8
testando nos valores  0  e  9
testando nos valores  1  e  0
testando nos valores  1  e  1
testando nos valores  1  e  2
testando nos valores  1  e  3
testando nos valores  1  e  4
testando nos valores  1  e  5
testando nos valores  1  e  6
testando nos valores  1  e  7
testando nos valores  1  e  8
testando nos valores  1  e  9
testando nos valores  2  e  0
testando nos valores  2  e  1
testando nos valores  2  e  2
testando nos valores  2  e  3
testando nos valores  2  e  4
testando nos valores  2  e  5
testando nos valores  2  e  6
testando nos valores  2  e  7
testando nos valores  2  e  8
testando nos valores  2  e  9
testando nos valores  3  e  0
testando nos valores  3  e  1
testando nos valores  3  e  2
testando n